# Projection to the z-space

References:
-   z-space: https://github.com/minyoungg/pix2latent/issues/3
-   w-space: https://github.com/woctezuma/stylegan2-projecting-images

## Requirements for alignment

### Switch to Tensorflow 1.x

In [1]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


### Install my fork of StyleGAN2

Reference: https://github.com/woctezuma/stylegan2/tree/tiled-projector

In [2]:
%cd /content/
!git clone https://github.com/woctezuma/stylegan2.git

%cd /content/stylegan2/
!git checkout tiled-projector

/content
fatal: destination path 'stylegan2' already exists and is not an empty directory.
/content/stylegan2
Already on 'tiled-projector'
Your branch is up to date with 'origin/tiled-projector'.


### Download model (face landmarks)

Reference: http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2

In [3]:
%mkdir -p /root/.keras/temp/

%cd /root/.keras/temp/
!wget https://github.com/davisking/dlib-models/raw/master/shape_predictor_68_face_landmarks.dat.bz2

/root/.keras/temp
--2020-09-22 19:11:29--  https://github.com/davisking/dlib-models/raw/master/shape_predictor_68_face_landmarks.dat.bz2
Resolving github.com (github.com)... 192.30.255.113
Connecting to github.com (github.com)|192.30.255.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/davisking/dlib-models/master/shape_predictor_68_face_landmarks.dat.bz2 [following]
--2020-09-22 19:11:29--  https://raw.githubusercontent.com/davisking/dlib-models/master/shape_predictor_68_face_landmarks.dat.bz2
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64040097 (61M) [application/octet-stream]
Saving to: ‘shape_predictor_68_face_landmarks.dat.bz2.4’

shape_predictor_68_ 100%[===================>]  61.07M   

## Alignment

### Download

In [4]:
%mkdir -p /content/stylegan2/raw_images/

%cd /content/stylegan2/raw_images/
!wget https://upload.wikimedia.org/wikipedia/commons/b/b0/Venus_botticelli_detail.jpg

/content/stylegan2/raw_images
--2020-09-22 19:11:30--  https://upload.wikimedia.org/wikipedia/commons/b/b0/Venus_botticelli_detail.jpg
Resolving upload.wikimedia.org (upload.wikimedia.org)... 198.35.26.112, 2620:0:863:ed1a::2:b
Connecting to upload.wikimedia.org (upload.wikimedia.org)|198.35.26.112|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 992216 (969K) [image/jpeg]
Saving to: ‘Venus_botticelli_detail.jpg.5’

Venus_botticelli_de 100%[===================>] 968.96K  6.07MB/s    in 0.2s    

2020-09-22 19:11:30 (6.07 MB/s) - ‘Venus_botticelli_detail.jpg.5’ saved [992216/992216]



### Align

In [5]:
%cd /content/stylegan2/
!python align_images.py raw_images/ aligned_images/

/content/stylegan2


## Requirements for projection

### Install pix2latent

Reference: https://github.com/minyoungg/pix2latent

In [6]:
%cd /content/
!git clone https://github.com/minyoungg/pix2latent.git

%cd /content/pix2latent/
%pip install -r requirements.txt
%pip install .

/content
fatal: destination path 'pix2latent' already exists and is not an empty directory.
/content/pix2latent
Processing /content/pix2latent
  Created wheel for pix2latent: filename=pix2latent-0.0.1-cp36-none-any.whl size=73701 sha256=2ce923156fbe0b0351786fe20d2fe2dab6b2eb022a83343dd4672bf77cc9028c
  Stored in directory: /tmp/pip-ephem-wheel-cache-00zxsizl/wheels/5c/3d/be/64951ba1ef2bdba4b70ca23355afe1556b61e3a5f8c2e724c6
Successfully built pix2latent
  Found existing installation: pix2latent 0.0.1
    Uninstalling pix2latent-0.0.1:
      Successfully uninstalled pix2latent-0.0.1


### Install additional packages

Reference: https://github.com/rosinality/stylegan2-pytorch

In [7]:
%pip install ninja

## Projection

Reference: https://github.com/minyoungg/pix2latent/blob/master/examples/

In [8]:
%cd /content/pix2latent/examples/

/content/pix2latent/examples


### Imports & argument parser

In [20]:
import os, os.path as osp
import numpy as np
import argparse

import torch
import torch.nn as nn

from pix2latent.model.stylegan2 import StyleGAN2

from pix2latent import VariableManager, save_variables
from pix2latent.optimizer import NevergradOptimizer
from pix2latent.utils import image, video

import pix2latent.loss_functions as LF
import pix2latent.utils.function_hooks as hook
import pix2latent.distribution as dist


parser = argparse.ArgumentParser()
parser.add_argument('--ng_method', type=str, default='CMA')
parser.add_argument('--lr', type=float, default=0.05)
parser.add_argument('--latent_noise', type=float, default=0.05)
parser.add_argument('--truncate', type=float, default=2.0)
parser.add_argument('--make_video', action='store_true')
parser.add_argument('--num_samples', type=int, default=4)
parser.add_argument('--max_minibatch', type=int, default=9)

_StoreAction(option_strings=['--max_minibatch'], dest='max_minibatch', nargs=None, const=None, default=9, type=<class 'int'>, choices=None, help=None, metavar=None)

### Inputs

In [21]:
args = parser.parse_args([])

In [22]:
model_name = 'ffhq'

if model_name == 'cars':
  img_size = 512
else:
  img_size = 1024

In [23]:
filename = '/content/stylegan2/aligned_images/Venus_botticelli_detail_01.png'

fn = filename.split('/')[-1].split('.')[0]
save_dir = f'./results/stylegan2_ffhq/ng_{args.ng_method}_{fn}'

### Initialize

In [25]:
### ---- initialize --- ###

model = StyleGAN2(model=model_name, search='z')

target = image.read(filename, as_transformed_tensor=True, im_size=img_size,
                    transform_style='stylegan')

loss_mask = torch.zeros((3, img_size, img_size))

if model_name == 'cars':
  # we apply a mask since the generated resolution is 384 x 512
  loss_mask[:, 64:-64, :].data += 1.0
else:
  loss_mask.data += 1.0

weight = loss_mask






model = StyleGAN2(model=model_name, search='z')
model = nn.DataParallel(model)
loss_fn = LF.ProjectionLoss()


var_manager = VariableManager()

var_manager.register(
                variable_name='z',
                shape=(512,),
                default=None,
                grad_free=True,
                distribution=dist.TruncatedNormalModulo(
                                            sigma=1.0,
                                            trunc=args.truncate
                                            ),
                var_type='input',
                learning_rate=args.lr,
                hook_fn=hook.Compose(
                            hook.NormalPerturb(sigma=args.latent_noise),
                            hook.Clamp(trunc=args.truncate),
                            )
                )

var_manager.register(
                variable_name='target',
                shape=(3, img_size, img_size),
                requires_grad=False,
                default=target,
                var_type='output'
                )

var_manager.register(
                variable_name='weight',
                shape=(3, img_size, img_size),
                requires_grad=False,
                default=weight,
                var_type='output'
                )

var_manager.register(
                variable_name='loss_mask',
                shape=(3, img_size, img_size),
                requires_grad=False,
                default=loss_mask,
                var_type='output'
                )

Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [on]
Loading model from: /usr/local/lib/python3.6/dist-packages/lpips/weights/v0.1/alex.pth


True

### Optimize

**Caveat**: Google Colab does not offer enough RAM to run CMA or BasinCMA on FFHQ data.

In [26]:
### ---- optimize --- ###

opt = NevergradOptimizer(
            args.ng_method, model, var_manager, loss_fn,
            max_batch_size=args.max_minibatch,
            log=args.make_video
            )

opt.log_resize_factor = 0.25

vars, out, loss = opt.optimize(
            num_samples=args.num_samples, meta_steps=1000, grad_steps=300
            )

(11_w,22)-aCMA-ES (mu_w=6.5,w_1=26%) in dimension 512 (seed=<module 'time' (built-in)>, Tue Sep 22 20:01:43 2020)
(optimize) progress 4% [50/1300] (0.690 sec/iter)
(optimize) progress 8% [100/1300] (0.706 sec/iter)
(optimize) progress 12% [150/1300] (0.717 sec/iter)
(optimize) progress 15% [200/1300] (0.727 sec/iter)
(optimize) progress 19% [250/1300] (0.732 sec/iter)
(optimize) progress 23% [300/1300] (0.740 sec/iter)
(optimize) progress 27% [350/1300] (0.745 sec/iter)
(optimize) progress 31% [400/1300] (0.751 sec/iter)
(optimize) progress 35% [450/1300] (0.752 sec/iter)
(optimize) progress 38% [500/1300] (0.752 sec/iter)
(optimize) progress 42% [550/1300] (0.754 sec/iter)
(optimize) progress 46% [600/1300] (0.754 sec/iter)
(optimize) progress 50% [650/1300] (0.757 sec/iter)
(optimize) progress 54% [700/1300] (0.757 sec/iter)
(optimize) progress 58% [750/1300] (0.756 sec/iter)
(optimize) progress 62% [800/1300] (0.755 sec/iter)
(optimize) progress 65% [850/1300] (0.755 sec/iter)
(opti

### Save results

In [27]:
### ---- save results ---- #

vars.loss = loss
os.makedirs(save_dir, exist_ok=True)

save_variables(osp.join(save_dir, 'vars.npy'), vars)

if args.make_video:
    video.make_video(osp.join(save_dir, 'out.mp4'), out)

image.save(osp.join(save_dir, 'target.jpg'), target)
image.save(osp.join(save_dir, 'mask.jpg'), image.binarize(weight))
image.save(osp.join(save_dir, 'out.jpg'), out[-1])
np.save(osp.join(save_dir, 'tracked.npy'), opt.tracked)

### Archive results before downloading them

In [30]:
!du -sh results/stylegan2_ffhq/*/*

20K	results/stylegan2_ffhq/ng_CMA_Venus_botticelli_detail_01/mask.jpg
3.6M	results/stylegan2_ffhq/ng_CMA_Venus_botticelli_detail_01/out.jpg
1.2M	results/stylegan2_ffhq/ng_CMA_Venus_botticelli_detail_01/target.jpg
19M	results/stylegan2_ffhq/ng_CMA_Venus_botticelli_detail_01/tracked.npy
145M	results/stylegan2_ffhq/ng_CMA_Venus_botticelli_detail_01/vars.npy


In [ ]:
# !tar --exclude vars.npy -cvf pix2latent_output.tar.gz results/stylegan2_ffhq/
# !du -sh pix2latent_output.tar.gz